In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!unzip -qq "/content/gdrive/MyDrive/제주도.zip"

In [3]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv('/content/open/train.csv')
test = pd.read_csv('/content/open/test.csv')

In [5]:
train

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [6]:
dataset = pd.DataFrame()

In [7]:
dataset['item'] = train['item']
dataset['corporation']=train['corporation']
dataset['location']=train['location']
dataset['price(원/kg)']=train['price(원/kg)']

In [8]:
dataset

,item,corporation,location,price(원/kg)
0,TG,A,J,0.0
1,TG,A,J,0.0
2,TG,A,J,1728.0
3,TG,A,J,1408.0
4,TG,A,J,1250.0
...,...,...,...,...
59392,RD,F,J,468.0
59393,RD,F,J,531.0
59394,RD,F,J,574.0
59395,RD,F,J,523.0


In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
dataset['item'] = label_encoder.fit_transform(dataset['item'])
dataset['corporation'] = label_encoder.fit_transform(dataset['corporation'])
dataset['location'] = label_encoder.fit_transform(dataset['location'])

In [10]:
dataset

,item,corporation,location,price(원/kg)
0,4,0,0,0.0
1,4,0,0,0.0
2,4,0,0,1728.0
3,4,0,0,1408.0
4,4,0,0,1250.0
...,...,...,...,...
59392,3,5,0,468.0
59393,3,5,0,531.0
59394,3,5,0,574.0
59395,3,5,0,523.0


In [11]:
dataset

,item,corporation,location,price(원/kg)
0,4,0,0,0.0
1,4,0,0,0.0
2,4,0,0,1728.0
3,4,0,0,1408.0
4,4,0,0,1250.0
...,...,...,...,...
59392,3,5,0,468.0
59393,3,5,0,531.0
59394,3,5,0,574.0
59395,3,5,0,523.0


In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)

scaled_data

array([[1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.08264384],
       ...,
       [0.75      , 1.        , 0.        , 0.02745229],
       [0.75      , 1.        , 0.        , 0.02501315],
       [0.75      , 1.        , 0.        , 0.02530011]])

In [13]:
type(scaled_data)

numpy.ndarray

In [14]:
scaled_data.shape

(59397, 4)

In [15]:
len(scaled_data)

59397

In [16]:
train_x = []
train_y = []

In [17]:
for i in range(len(scaled_data)):
    train_x.append(scaled_data[i:i+1, 0:3])
    train_y.append(scaled_data[i, 3])


In [18]:
train_x[0].shape

(1, 3)

In [19]:
len(train_x)

59397

In [20]:
len(train_y)

59397

In [21]:
train_x, train_y = np.array(train_x), np.array(train_y)

In [22]:
train_x.shape

(59397, 1, 3)

In [23]:
train_y.shape

(59397,)

In [24]:
train_x = np.reshape(train_x,(train_x.shape[0], train_x.shape[1],3))

In [25]:
train_x.shape

(59397, 1, 3)

In [26]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(128,return_sequences=True, input_shape=(train_x.shape[1],3)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [27]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [28]:
model.fit(train_x, train_y, batch_size=100, epochs=10)

Epoch 1/10
594/594 [==============================] - 14s 8ms/step - loss: 0.0075
Epoch 2/10
594/594 [==============================] - 4s 6ms/step - loss: 0.0061
Epoch 3/10
594/594 [==============================] - 3s 6ms/step - loss: 0.0061
Epoch 4/10
594/594 [==============================] - 3s 6ms/step - loss: 0.0060
Epoch 5/10
594/594 [==============================] - 4s 7ms/step - loss: 0.0060
Epoch 6/10
594/594 [==============================] - 4s 7ms/step - loss: 0.0060
Epoch 7/10
594/594 [==============================] - 3s 6ms/step - loss: 0.0060
Epoch 8/10
594/594 [==============================] - 3s 5ms/step - loss: 0.0060
Epoch 9/10
594/594 [==============================] - 3s 6ms/step - loss: 0.0060
Epoch 10/10
594/594 [==============================] - 7s 11ms/step - loss: 0.0059


In [29]:
test

,ID,timestamp,item,corporation,location
0,TG_A_J_20230304,2023-03-04,TG,A,J
1,TG_A_J_20230305,2023-03-05,TG,A,J
2,TG_A_J_20230306,2023-03-06,TG,A,J
3,TG_A_J_20230307,2023-03-07,TG,A,J
4,TG_A_J_20230308,2023-03-08,TG,A,J
...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J
1088,RD_F_J_20230328,2023-03-28,RD,F,J
1089,RD_F_J_20230329,2023-03-29,RD,F,J
1090,RD_F_J_20230330,2023-03-30,RD,F,J


In [30]:
dataset_test = pd.DataFrame()

In [31]:
dataset_test['item'] = test['item']
dataset_test['corporation']=test['corporation']
dataset_test['location']=test['location']

In [32]:
dataset_test

,item,corporation,location
0,TG,A,J
1,TG,A,J
2,TG,A,J
3,TG,A,J
4,TG,A,J
...,...,...,...
1087,RD,F,J
1088,RD,F,J
1089,RD,F,J
1090,RD,F,J


In [33]:
dataset_test['item'] = label_encoder.fit_transform(dataset_test['item'])
dataset_test['corporation'] = label_encoder.fit_transform(dataset_test['corporation'])
dataset_test['location'] = label_encoder.fit_transform(dataset_test['location'])

In [34]:
dataset_test

,item,corporation,location
0,4,0,0
1,4,0,0
2,4,0,0
3,4,0,0
4,4,0,0
...,...,...,...
1087,3,5,0
1088,3,5,0
1089,3,5,0
1090,3,5,0


In [35]:
test_x=[]

In [36]:
scaled_test_data=scaler.fit_transform(dataset_test)

In [37]:
scaled_test_data

array([[1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       ...,
       [0.75, 1.  , 0.  ],
       [0.75, 1.  , 0.  ],
       [0.75, 1.  , 0.  ]])

In [38]:
scaled_test_data.shape

(1092, 3)

In [39]:
type(scaled_test_data)

numpy.ndarray

In [40]:
for i in range(len(scaled_test_data)):
    test_x.append(scaled_data[i:i+1, 0:3])

In [41]:
test_x = np.array(test_x)

In [42]:
test_x.shape

(1092, 1, 3)

In [43]:
predictions = model.predict(test_x)

35/35 [==============================] - 1s 3ms/step


In [44]:
predictions

array([[0.11495236],
       [0.11495236],
       [0.11495236],
       ...,
       [0.11495236],
       [0.11495236],
       [0.11495236]], dtype=float32)

In [45]:
predictions[0]

array([0.11495236], dtype=float32)

In [46]:
predictions.shape

(1092, 1)

In [47]:
predictions[0]

array([0.11495236], dtype=float32)

In [48]:
predictions.shape

(1092, 1)

In [49]:
predictions

array([[0.11495236],
       [0.11495236],
       [0.11495236],
       ...,
       [0.11495236],
       [0.11495236],
       [0.11495236]], dtype=float32)

In [50]:
predictions = predictions.reshape(-1, 3)

In [51]:
predictions = scaler.inverse_transform(predictions)

In [293]:
predictions[0]

array([0.60372746], dtype=float32)

In [55]:
# 배열을 평탄화하고 (344*3, 1) 형태로 재구성
predictions = predictions.flatten().reshape(1092, 1)

In [56]:
sub = pd.read_csv('/content/open/sample_submission.csv')

In [57]:
sub['answer'] =predictions

In [58]:
sub

,ID,answer
0,TG_A_J_20230304,0.459809
1,TG_A_J_20230305,0.574762
2,TG_A_J_20230306,0.114952
3,TG_A_J_20230307,0.459809
4,TG_A_J_20230308,0.574762
...,...,...
1087,RD_F_J_20230327,0.574762
1088,RD_F_J_20230328,0.114952
1089,RD_F_J_20230329,0.459809
1090,RD_F_J_20230330,0.574762


In [59]:
sub.to_csv('./baseline_submission.csv', index=False)